<a href="https://colab.research.google.com/github/RegaipKURT/QuantumProgramming/blob/master/Bernstein_Vazirani_Algoritmas%C4%B1_ve_Kuantum_Bilgisayarlar%C4%B1n_H%C4%B1z%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bernstein-Vazirani Algoritması ve Kuantum Bilgisayarların Hızı

#### Bir kutunun içinde gizli bir sayı olduğunu ve bu sayıyı bir bilgisayarla bulmaya çalıştığımızı düşünelim. Bu numara 6-bite karşılık gelecek şekilde tanımlanmış olsun. *(6 tane 0 ve 1'in yan yana gelmesi ile 2'lik tabanda yazılmış bir sayı.)* Klasik bilgisayarlar bu 6-bitlik sayıyı bulmak için 6 deneme yapmak zorundadırlar. (N bitlik bir sayı için N sayıda deneme!) Fakat bir quantum bilgisayar ve Bernstein-Vazirani algoritması kullanarak bu sayıyı tek bir denemede (veya işlemde) bulabilirsiniz. 

* Normalde programlama dillerinde integer (int) bir değer tanımladığımızda çoğu dilde 32 bitlik bir sayı tanımlanır. 32 bit 2^32 tane değer alabilir demektir. Bunların yarısı negatif, 0 ise pozitiflere dahil olduğu için: toplamda ( (2^32)/2 ) - 1 tane pozitif değer tanımlayabiliriz. 

##### Yani 6 bitlik bir sayı klasik bir bilgisayarda 2^6 tane değer barındırabilir. 

#### Konuyu bir örnekle açıklayalım:

2'lik sayı tabanında 101001 (10'luk tabanda 41 sayısına karşılık gelir. Negatif sayılar dahil edilmediğinde 0-64 arası sayılar 6 tane bit ile yazılabilir.) sayısını tahmin etmeye çalıştığımızı varsayalım. Klasik bilgisayarlar bu 101001 sayısını 6 denemede şöyle bulabilirler.


# Klasik bilgisayar bu sayıyı en kısa şekilde nasıl bulabilir?

1 0 1 0 0 1  ===> bitlerin içinde tutulan bilmediğimiz sayı

0 0 0 0 0 1  ===> bilgisayarın ilk tahmini. Ardından bu sayıları and mantıksal operatörüne sokalım: 

1. deneme:

* Asıl sayı: 1 0 1 0 0 1 
* Tahmin :   0 0 0 0 0 1
* And Op.:   0 0 0 0 0 1

##### Son basamağı 1 yapıp and operatörü (mantıksal ve işlemi) uyguladığımızda sonuç 1 dönüyorsa orjinal sayının da son basamağı 1 demektir. İlk basamağımızı bulduk, sonuç 1.

2. deneme:

* Asıl sayı: 1 0 1 0 0 1 
* Tahmin :   0 0 0 0 1 0
* And Op.:   0 0 0 0 0 0 

##### Şimdi tahminimizde sadece sondan 1 önceki basamağın değerini 1 ve diğerlerini sıfır yapalım. And operatörü uygulandığında sondan bir önceki basamak için sonuç 0 dönecek. Demek ki sondan 1 önceki basamağımız da 0. 

* Bu şekilde sırayla her seferinde bulmak istediğimiz basamağı 1 diğerlerini sıfır yaparsak ve orjinal sayı ile tahmin üzerinde and operatörü uygularsak sonucu bulmuş oluruz. Her denemede sadece orjinal sayının da aynı basamağı 1 ise and işleminin sonucu 1 döner. Aksi takdirde 0'dır. 

##### Klasik bilgisayarların bu işlemi yapmasının yolu budur. 
#### NOT: 6 bitlik bir kutunun içinde 64 tane sayı olabilmesine rağmen klasik bir bilgisayarda and operatörü kullanarak 6 denemede bu sayıyı bulduk. 64 sayıyı da ard arda denememize gerek yoktur.

# Kuantum Bilgisayarın Tahmin İşlemi

#### Yukarıda gördük ki N bitlik (2^N değer alabilen) bir sayıyı klasik bilgisayarlar N adımda tahmin edebiliyor. Fakat bundan daha etkileyici olan Kuantum Bilgisayarlar bunu sadece 1 adımda yapabilir. 

#### **Şimdi nasıl olacağına bakalım!**


# Kütüphanelerin İmport Edilmesi

In [0]:
!pip install qiskit # qiskit kütüphanesi yüklü değilse yükleyelim!!!

In [0]:
from qiskit import *
from qiskit.tools.visualization import plot_bloch_multivector, plot_histogram

%matplotlib inline

#### Yapacaklarımızı sırayla anlatıp daha sonra işlemlere geçelim.
* İlk önce 7 tane qubit ve 6 tane klasik bit ile bir kuantum devresi oluşturacağız. 7 tane qubit olmasının sebebi 7. qubit'i yukarıdaki and operatöründe kullandığımız gibi orjinal değerleri belirlemek için kullanacağız. 

* Daha sonra sayımızı içeren ilk 6 qubit'i süperpozisyon durumuna 7. qubit'i ise x kapısı ile 1 durumuna getireceğiz.

* Ardından orjinal sayımızda 1 olan qubit değerlerine cx (controlled not) uygulayacağız. Böylece sadece 1 olan yerlerde 1, 0 olan yerlerde 0 değerini almış olacağız.

* Son olarak okuduğumuz değeri klasik bitlere yazdırıp sonucu alabiliriz! 

# QİSKİT İLE KODLARIN YAZILMASI

Gizli numaramızı belirleyelim. Daha sonra bir kuantum devresi aracılığıyla bu numarayı bulmaya çalışacağız.

In [0]:
secret_number = "101001" # gizli numaramız yukarıda bahsettiğimiz sayıyla aynı olsun.

Aşağıda bir QuantumCircuit() fonksiyonuyla bir kuantum devresi oluşturacağız. Bütün kodlarımız bu devrenin içinde yer alacak. QuantumCircuit() fonksiyonunun iki parametre alacak:

* 1. Parametre: Qubit sayımız; 7 olacak, 6 tanesinde sayımızın olduğunu düşüneceğiz ve 7. si ise bir nevi turnusol kağıdı vazifesi görerek bunların değerlerini bulacak. 
* 2. Parametere: Kullanılacak klasik bitlerin sayısıdır. Klasik bitleri kullanmamızın tek sebebi, sonucu aldığımızda bu bitlerin üzerine yazıp okumaktır. Başka bir amaçla kullanmayacağız şimdilik. Bu yüzden de 6 tane klasik bit tanımlayacağız, çünkü sayımız 6 tane bite karşılık geliyor ve değerleri işlemin sonunda bu bitlerin üzerine yazılacak.

In [0]:
circuit = QuantumCircuit(len(secret_number)+1, len(secret_number)) # 7 tane qubit, 6 tane klasik bit.

Aşağıda hadamard kapısı ile bu ilk 6 qubiti yani süperpozisyon durumuna getireceğiz. Süperpozisyon bir qubit'in aynı anda hem 0 hem de 1 değerlerine sahip olabilmesi durumudur! 

In [0]:
# şimdi ilk 4 qubite hadamard kapısı uygulayıp süperpozisyon durumuna getirelim
circuit.h(range(len(secret_number)))

**Pauli kapısı veya x kapısı:** İçine aldığı qubit'in değerini 1 yapar. 7. qubitin değerini 1 yapmak sayımızı içeren qubitlerin değerlerini bulmaya yarayacak.

In [0]:
#şimdi 7. qubit üzerinde pauli (x) ve hadamard kapıları uygulayalım!
circuit.x(len(secret_number))
circuit.h(len(secret_number))

circuit.barrier() #barrier uygulamak görselleştirme açısından kolaylık sağlıyor sadece!

Şimdi yukarıda turnusol kağıdı vazifesi görecek dediğimiz 7. qubit ile, sayımız arasında bir ilişki kuralım. Bu ilişkiyi kurmak için cx() fonksiyonunu kullanacağız. cx fonksiyonu 2 parametre alır:
* 1) Kontrol biti: Kontrol edilecek qubit
* 2) Hedef bit: kontrol sonucunda üzerinde işlem yapılacak bit.

Cx fonksiyonu içine aldığı kontrol bitinin değeri 1 ise hedef bitin değerini de 1 yapar, değilse işlem yapmaz.

Gizli sayımızın içinde 1 değerinin olduğu her qubit'e cx fonksiyonu koyarak, 7. qubit ile karşılaştıracağız. Eğer ilgili qubit'te 1 değeri değeri varsa 7. qubit 1 değilse 0 değeri almış olacak. Böylece biz de 7. qubitin her bite göre aldığı değere bakarak sayımızı bulmuş olacağız!

In [0]:
# "101001" baştan sayarsak 5. 3. ve 0. bitler 1 olursa ve hedef bit 1 olur, aksi takdirde 0 olacaktır.

"""Aşağıda kod da aynı işi görür ve daha anlaşılır. Ama daha genel bir kod yazmak istiyoruz!"""
# circuit.cx(5,len(secret_number))
# circuit.cx(3,len(secret_number))
# circuit.cx(0,len(secret_number))
""" Yukarıdaki kodu da şöyle düzenleyelim: """
for i, deger in enumerate(reversed(secret_number)): 
    # ters çevirdik çünkü qiskit qubitleri aşağıdan yukarıya doğru sıralandırıyor!
    if deger == "1":
        circuit.cx(i, len(secret_number))

circuit.barrier()

Sayımızın olduğu qubitlerin hepsini tekrar süperpozisyon durumuna alalım

In [0]:
circuit.h(range(len(secret_number)))
circuit.barrier()

Her şey bitti. Şimdi ölçüm yapabiliriz. Ölçüm yapmak için measure fonksiyonu kullanılır. İlk parametresi ölçülecek qubitler, ikinci parametresi ise dönen değerlerin üzerine yazılacağı klasik bit'lerdir.

In [0]:
circuit.measure(range(len(secret_number)),range(len(secret_number)))

simulator = Aer.get_backend("qasm_simulator")
sonuc = execute(circuit, backend=simulator, shots=1).result()

### Yukarıda gördüğümüz yöntemlerin (hem kasik hem quantum bilgisayar için) tamamına **Bernstein-Vazirani Algoritması** deniyor.

## Algoritmanın Formülü: f:{0,1}n→{0,1} x→a⋅x+b(mod2) (a∈{0,1}n,b∈{0,1})

#### Aşağıda yazdığımız kodun tamamını draw fonksiyonuyla görselleştirelim. Grafik üzerinde kodu anlamak ilk başta biraz karmaşık gelebilir. Barrier koyduğumuz yerlerden ayırıp inceleyerek anlamaya çalışabilirsiniz.

In [0]:
circuit.draw(output="mpl")

![alt text](https://github.com/RegaipKURT/QuantumProgramming/blob/master/circuit.png?raw=true)

In [0]:
import matplotlib.pyplot as plt

plt.figure()
plt.bar(sonuc.get_counts().keys(), sonuc.get_counts().values())
plt.title("SONUÇLAR")
plt.ylabel("İhtimal")
plt.xlabel("Sonuç")
plt.show()

![alt text](https://github.com/RegaipKURT/QuantumProgramming/blob/master/plot.png?raw=true)

# Gerçek bir kuantum bilgisayarda çalıştırmak!

###IBM'in internet üzerinden bize sağladığı api key ile (IBMQ üzerinden alabilirsiniz) gerçek bir quantum bilgisayar üzerinde kodlarımızı internetten göndererek çalıştırabiliriz. 

#### Aşağıda yazdığımız kodla önce kullanabileceğimiz kuantum bilgisayarlarının hangileri olduğuna baktık ve ardından da uygun bir bilgisayar belirleyerek o bilgisayar üzerinde çalışmasını sağladık. 

#### Fakat 14 qubit içeren bilgisayara ulaşmakta sıkıntı yaşadığım için 5 qubit olan bilgisayarı kullandım. Bunun için de ilk başta kullandığımız 6 bitlik sayıyı 4 bitlik bir sayıyla değiştirdim.

#### Sonuç her ne kadar doğru da olsa aşağıda bize başka ihtimaller de dönecek. Yine de en yüksek ihtimale sahip sayımız 1110 olacaktır. 




In [0]:
IBMQ.save_account(open("token.txt", "r").read(), overwrite=True) 
#Kendi api key'ininizi kopyalayın, ben dosyadan okuttum!
IBMQ.load_account()

provider = IBMQ.get_provider("ibm-q")

for backend in provider.backends():
    try:
        qubit_count = len(backend.properties().qubits)
    except:
        qubit_count = "simulated"

    print(f"{backend.name()} has {backend.status().pending_jobs} qued and {qubit_count} qubits.")

In [0]:
"""-------------------------Aşağıda kodumuzu aynı şekilde yazdık!---------------------------"""
secret_number = "1110" 

circuit = QuantumCircuit(len(secret_number)+1, len(secret_number)) # 5 tane qubit, 4 tane klasik bit.

# şimdi ilk 4 qubite hadamard kapısı uygulayıp süperpozisyon durumuna getirelim
circuit.h(range(len(secret_number)))

#şimdi 5. qubit üzerinde pauli (x) ve hadamard kapıları uygulayalım!
circuit.x(len(secret_number))
circuit.h(len(secret_number))

circuit.barrier()

for i, deger in enumerate(reversed(secret_number)):
    if deger == "1":
        circuit.cx(i, len(secret_number))

circuit.barrier()

circuit.h(range(len(secret_number)))
circuit.barrier()

circuit.measure(range(len(secret_number)),range(len(secret_number)))
"""-----------------------------------------------------------------------"""

from qiskit.tools.monitor import job_monitor

backend = provider.get_backend("ibmq_essex") # yukarıdan bir bilgisayar seçtik işlem için
job = execute(circuit, backend=backend, shots=500) # bu sefer tek bir işlem değil 500 deneme yaptık.
job_monitor(job)

#### Yukarıda shots değerini 500'e ayarlamamız sizi yanıltmasın. Bunu yapmamızın sebebi gürültü diye bahsettiğimiz durumu göstermek ve bu durumdan kurtulmaktı. Yoksa kusursuz bir quantum bilgisayarı tek adımda bu işlemi yapabilir.

# Sonuçları Görselleştirelim

In [0]:
import matplotlib.pyplot as plt

sonuc = job.result()

plt.figure()
plt.bar(sonuc.get_counts().keys(), sonuc.get_counts().values())
plt.title("SONUÇLAR")
plt.ylabel("İhtimal")
plt.xlabel("Sonuç")
plt.show()

![alt text](https://github.com/RegaipKURT/QuantumProgramming/blob/master/qplot.png?raw=true)

## Kuantum Gürültüsü

#### Peki neden başka ihtimaller de döndü diye sorabiliriz. Bunun sebebi şu an elimizde ***kusursuz çalışan bir kuantum bilgisayarı olmamasıdır.*** Kuantum bilgisayarlar dış etkilere karşı muazzam derecede hassastır. Bazen çalışan diğer işlemlerden, bazen nemden ve sıcaklıktan, bazen de başka sebeplerden kuantum bilgisayarların çalışması etkilenir. Bundan dolayı ortaya çıkan sonuçta beklenmeyen ihtimaller de görülebilir. Buna *gürültü* diyoruz. Sonuçta yaptığımız işlem çok hassas olmadığı için istediğimiz çıktıyı yine de alabildik. 

#### Ama eğer siz ben bu gürültüyü istemiyorum diyorsanız, ilk başta yaptığımız gibi simülasyon üzerinde de çalışabilirsiniz. qasm simülatörü (quantum assembly'nin kısaltılmışı) kusursuz bir quantum bilgisayarını temsil edecek şekilde çalışır. Dolayısıyla qasm_simulator backend'i üzerinde aldığımız sonuçlar kusursuz bir kuantum bilgisayarıyla aynı sonuçlardır. Tek fark kodumuzu kendi bilgisayarımız üzerinde çalıştırmasıdır.

# EEE, NE FARK VAR ŞİMDİ?

Klasik bilgisayar için açıkladığımız işlemle, kuantum bilgisayarı üzerinde yaptığımız işlem size çok benzer hatta neredeyse aynı gibi gelmiş olabilir. Ama iki bilgisayarın yaptığı iş arasında şöyle bir fark var: 

#### Normal bilgisayar and operatörünü kullanabilmek için her seferinde bir işlem yapmak zorundadır. Çünkü her bir karşılaştırma için ayrı ayrı, işlemcideki basit mantık kapılarından biri olan and operatörünü kullanması gerekmektedir. Dolayısıyla da basamak sayısı kadar işlemciyi tekrar çalıştırması gerekecektir. 

#### Ama bizim yazdığımız kod bir bütün olarak kuantum devresidir. O yüzden ingilizce karşılı olan circuit şeklinde isimlendirilmiş zaten. Ve bu kuantum devresinin 2 özelliği var, 
* 1)  Bu devre kuantum bir işlemci üzerinden tek bir işlemde çıktı verir.
* 2) Bu işemi normal bilgisayardan daha hızlı yapmasıdır. 

#### Fakat burda kastedilen hız sadece zaman açısından yani 2. bashsettiğimiz özellik değil. Ki bu çok öenmli bir farklılık da değil. ***Asıl önemli olan 1. farklılıktır. Çünkü bizim örneğimiz üzerinden bakacak olursak N kaç olursa olsun işlem sadece bir adımda tamamlanır. Yani işlem sayısı N'den bağımsızdır.*** Kuantum bilgisayarların devrim yaptığı nokta da burasıdır.

# SONUÇ

Buradan çıkaracağımız ilk sonuç kuantum bilgisayarların, klasik bilgisayarların 2^n adımda yaptığı ***bazı işleri*** sadece n adımda yapabildiğidir. ***Bazı işleri*** diyorum çünkü bu klasik bilgisayarlardaki her işlem için geçerli değil. Yukarıda yaptığımız örnek buna uygun olduğu için kuantum bilgisayarında yapabildik. Ama bu herşeyi böyle halledebileceğimizi göstermez.

##### Sonuçlarda da görebileceğimiz gibi kuantum bilgisayarların geri dönüş değeri istatistiksel olasılık temeline dayanır. Yeri gelmişken belirtelim ki kuantum fiziğinde bu olasılıkları içeren matrislere durum matrisleri denir (Psi-Bra-Ket durum matrisleri gibi!) ve bütün durum matrisleri normalize edilebilrdir. Sebebi de istatistiksel olarak bütün olasılıkların toplamının 1'e eşit olması zorunluluğudur. Bunları başka örneklerimizde göreceğiz.

* Yukarıda biz 7 bitlik bir sayıyı tahmin etmeye çalıştık. Bu bizim kullandığımız verimli bir algoritma ile klasik bilgisayarda 7 adım gerektirir. Eğer and operatörü aracılığıyla böyle verimli bir algoritma kullanmasaydık, en kötü durumda 2^7 tane tahmin yapmamız gerekecekti.

* Kuantum bilgisayarında kullandığımız algoritma 7 bitlik bir sayıyı tek işlemde buldu. Bu da Bernstein-Vazirani aldoritması aracılığıyla mümkün oldu. Üstelik 7 değil 777 bitlik bir sayı da tanımlasak yine tek adımda işlem yapılmış olacaktı. Kuantum bilgisayar açısından bir şey farketmezdi.

* Yukarıdaki durum için kuantum bilgisayar ile klasik bilgisayarın işlem sayısı arasında n-1 işlem sayısı fark var. Yani 5 işlemlik bir verim farkı. Fakat n büyüdükçe sorun artacaktır. Üstelik bir klasik bilgisayar her zaman and operatörü ile kolayca tahmin yapamayabilir. Bu durumda da 2^n tane ihtimali denemesi gerekir.

* Klasik bilgisayarların 2^n ihtimali deneyeceği durumlarda gereken işlem sayısı üssel olarak artacağından bir yerden sonra işlem gücü yetersiz olacaktır. Bu durumlarda kuantum bilgisayarlar 1 adımda klasik bilgisayarın 2^n adımda yaptığı bir işlemi yapabilir. (veya bazen de n adıma da çıkabilir.)

* Bu örnek kuantum bilgisayarların her işlemi 1 adımda yaptığı anlamına gelmiyor, klasik bilgisayarda 2^n işlemde yapılan iş, kuantum bilgisayarda bazen 1 bazen de n işlemde yapılabilir. Hatta bazı durumlarda kuantum bilgisayarı klasik bilgisayardan daha yavaş olabilir ve zaten olacaktır da.

* Sonuç olarak kuantum bilgisayarların hızı işlemci hızı anlamındaki bir hız değildir. İstatistiksel olasılıkların ölçülmesi konusunda bir hızdır bu.

## En nihayetinde kuantum bilgisayarlar istatistik ve ölçüm temelli işlerde iyiyken, kesin matematiksel sonuç gerektiren işlerde klasik bilgisayarlar daha uygun ve daha hızlıdır.